In [2]:
import keras
import numpy as np
import keras.datasets
from keras.preprocessing import image_dataset_from_directory
from keras import Sequential, layers, models,callbacks, losses, optimizers, metrics
print(keras.__version__)

3.4.1


In [3]:
(x_train, y_train),(x_test, y_test) = keras.datasets.mnist.load_data()

In [4]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print("Train Label:", y_train)
print("Test Label:", y_test)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)
Train Label: [5 0 4 ... 5 6 8]
Test Label: [7 2 1 ... 4 5 6]


In [5]:
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
y_train shape: (60000,)
60000 train samples
10000 test samples


In [6]:
num_classes = 10
input_shape = (28, 28, 1)

In [22]:
model = keras.Sequential(
    [
        keras.layers.Input(shape=input_shape),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Dropout(0.25),

        keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Dropout(0.5),
        
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dense(num_classes, activation="softmax"),
    ]
)

In [23]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_13 (Conv2D)              │ (None, 26, 26, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 24, 24, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 22, 22, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 11, 11, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 11, 11, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 9, 9, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 7, 7, 64)       │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 3, 3, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 3, 3, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_4      │ (None, 64)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 222,794 (870.29 KB)

 Trainable params: 222,794 (870.29 KB)

 Non-trainable params: 0 (0.00 B)

In [24]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=[
        keras.metrics.SparseCategoricalAccuracy(name="acc"),
    ],
)

In [25]:
batch_size = 128
epochs = 10

callbacks = [
    keras.callbacks.ModelCheckpoint(filepath="model_at_epoch_{epoch}.keras"),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=2),
]

model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.15,
    callbacks=callbacks,
)
score = model.evaluate(x_test, y_test, verbose=0)

Epoch 1/10
399/399 ━━━━━━━━━━━━━━━━━━━━ 75s 183ms/step - acc: 0.6839 - loss: 0.9198 - val_acc: 0.9721 - val_loss: 0.0950
Epoch 2/10
399/399 ━━━━━━━━━━━━━━━━━━━━ 86s 216ms/step - acc: 0.9563 - loss: 0.1496 - val_acc: 0.9839 - val_loss: 0.0572
Epoch 3/10
399/399 ━━━━━━━━━━━━━━━━━━━━ 85s 213ms/step - acc: 0.9714 - loss: 0.0910 - val_acc: 0.9860 - val_loss: 0.0491
Epoch 4/10
399/399 ━━━━━━━━━━━━━━━━━━━━ 77s 194ms/step - acc: 0.9796 - loss: 0.0709 - val_acc: 0.9892 - val_loss: 0.0382
Epoch 5/10
399/399 ━━━━━━━━━━━━━━━━━━━━ 74s 185ms/step - acc: 0.9801 - loss: 0.0633 - val_acc: 0.9889 - val_loss: 0.0397
Epoch 6/10
399/399 ━━━━━━━━━━━━━━━━━━━━ 75s 189ms/step - acc: 0.9832 - loss: 0.0540 - val_acc: 0.9901 - val_loss: 0.0346
Epoch 7/10
399/399 ━━━━━━━━━━━━━━━━━━━━ 78s 179ms/step - acc: 0.9855 - loss: 0.0468 - val_acc: 0.9924 - val_loss: 0.0291
Epoch 8/10
399/399 ━━━━━━━━━━━━━━━━━━━━ 81s 177ms/step - acc: 0.9870 - loss: 0.0431 - val_acc: 0.9902 - val_loss: 0.0336
Epoch 9/10
399/399 ━━━━━━━━━━━━━

In [26]:
prediction = model.predict(x_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step


In [28]:
prediction

array([[2.66797540e-09, 8.80647361e-11, 9.88647571e-07, ...,
        9.99997377e-01, 2.49402321e-09, 7.81422557e-07],
       [2.55141208e-06, 5.98796703e-07, 9.99995828e-01, ...,
        1.41059111e-08, 6.19611527e-08, 2.91452484e-09],
       [1.82155276e-07, 9.99494195e-01, 8.14555108e-07, ...,
        3.71359609e-04, 2.91479719e-05, 1.34149914e-05],
       ...,
       [1.33447305e-14, 7.45040557e-11, 1.94776296e-11, ...,
        6.41647586e-12, 1.10540547e-07, 3.60068384e-08],
       [1.34403209e-07, 1.38715645e-13, 2.80545053e-12, ...,
        1.81137119e-10, 1.13190581e-04, 1.48641834e-06],
       [8.07130959e-07, 1.41908328e-12, 5.85213456e-05, ...,
        1.42105411e-17, 2.53448036e-07, 3.88553939e-10]], dtype=float32)